In [140]:
from langgraph.graph import StateGraph,START, END
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from typing import TypedDict,Annotated, Literal
from pydantic import BaseModel, Field
import operator

In [141]:
load_dotenv()

True

In [142]:
model = ChatGoogleGenerativeAI(model='gemini-2.5-pro')

In [143]:
class SentimentSchema(BaseModel):

    sentiment: Literal['positive', 'negative'] = Field(description='sentiment of the review')

In [144]:
class DiagnosisSchema(BaseModel):
    issue_type: Literal["UX", "Performance", "Bug", "Support", "Other"] = Field(description='The category of issue mentioned in the review')
    tone: Literal["angry", "frustrated", "disappointed", "calm"] = Field(description='The emotional tone expressed by the user')
    urgency: Literal["low", "medium", "high"] = Field(description='How urgent or critical the issue appears to be')

In [145]:
structured_model = model.with_structured_output(SentimentSchema)
structured_model_diagnosis = model.with_structured_output(DiagnosisSchema)

In [146]:
class SentiState(TypedDict):

    review: str

    sentiment: Literal['positive', 'negative']
    diagnosis: dict
    response: str

In [147]:
def find_sentiment(state: SentiState):

    prompt = f'for the following review find the sentiment \n {state["review"]}'
    sentiment = structured_model.invoke(prompt).sentiment

    return {'sentiment': sentiment}

def check_sentiment(state: SentiState) -> Literal['positive_response', 'run_diagnosis']:

    if state['sentiment'] == 'positive':
        return 'positive_response'
    else:
        return 'run_diagnosis'
    
def positive_response(state: SentiState):
    prompt = f'write a thank you message in response to this review \n {state["review"]}.  Do not explain or give multiple options. Just return a single message.'
    response = model.invoke(prompt).content
    return {'response': response}

def run_diagnosis(state: SentiState):

    prompt = f"""Diagnose this negative review:\n\n{state['review']}\n"
    "Return issue_type, tone, and urgency.
"""
    
    diagnosis = structured_model_diagnosis.invoke(prompt)

    return {'diagnosis': diagnosis.model_dump()}

def negative_response(state: SentiState):

    diagnosis = state['diagnosis']

    prompt = f"""You are a support assistant.
The user had a '{diagnosis['issue_type']}' issue, sounded '{diagnosis['tone']}', and marked urgency as '{diagnosis['urgency']}'.
Write an empathetic, helpful resolution message.  Do not explain or give multiple options. Just return a single message."""
    
    response = model.invoke(prompt).content

    return {"response": response}







In [148]:
graph = StateGraph(SentiState)

graph.add_node('find_sentiment', find_sentiment)
graph.add_node('check_sentiment', check_sentiment)
graph.add_node('positive_response', positive_response)
graph.add_node('run_diagnosis', run_diagnosis)
graph.add_node('negative_response', negative_response)


graph.add_edge(START, 'find_sentiment')
graph.add_conditional_edges('find_sentiment', check_sentiment)
graph.add_edge('positive_response', END)
graph.add_edge('run_diagnosis', 'negative_response')
graph.add_edge('negative_response', END)

workflow = graph.compile()

In [149]:
initial_state = {'review': 'i have never seen such a beautiful design like this and website performance is fast'}
final_state = workflow.invoke(initial_state)
print(final_state)

{'review': 'i have never seen such a beautiful design like this and website performance is fast', 'sentiment': 'positive', 'response': "Thank you so much for your incredibly kind words! We're thrilled that you love the design and are enjoying the fast performance, as that's exactly the experience we aim to create."}
